In [24]:
import sys
sys.path.insert(0, '/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/lib')
from capacityHelpers import *
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np

podName='na6'

pod = pd.read_csv('/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/lookups/podList.csv')
pod.set_index('pod', inplace=True)

numNodes = pod.loc[podName]['servers']
topNodes = numNodes - 1

file_name=podName+'-cpu-capacity.csv'
dir_name='/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/data/'
data = importArgusTimeSeriesCSV(dir_name+file_name)

data = data[~data.index.duplicated(keep='last')]

columnCount = len(data.columns)
rankLevel = topNodes/columnCount

rank = data.rank(axis=1, ascending=False, pct=True)
rank_boolean = data.rank(axis=1, ascending=False, pct=True) <= rankLevel
rank.to_csv('/Users/abertran/Documents/Results/ForecastResults/dbCPUUtil/rank.csv')
rank_boolean.to_csv('/Users/abertran/Documents/Results/ForecastResults/dbCPUUtil/rank_boolean.csv')
np.sort(pd.unique(rank.values.ravel('K')))
#(topNodes,columnCount)
#rankLevel
#rank_boolean.head()
rank_boolean.columns

Index(['db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-2}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-4}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-6}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-3}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-1}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-8}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-5}',
       'db.WAS.SP4.na6:CPUUtil{source=OEM,device=db1-7}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-2}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-4}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-6}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-3}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-1}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-8}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-5}',
       'db.CHI.SP4.na6:CPUUtil{source=OEM,device=db1-7}'],
      dtype='object')

In [106]:
column_list=rank_boolean.columns.tolist()

datacenter_sp=[i.split('.')[1]+'.'+i.split('.')[2] for i in q]
unique_dc_sp=list(set(datacenter_sp))
unique_dc_sp
df_dc1=rank_boolean.filter(regex=unique_dc_sp[0])
df_dc2=rank_boolean.filter(regex=unique_dc_sp[1])
data_dc1=data.filter(regex=unique_dc_sp[0])
data_dc2=data.filter(regex=unique_dc_sp[1])
column_list_datadc1=data_dc1.columns.tolist()
column_list_datadc2=data_dc2.columns.tolist()
data_dc1['mode']=df_dc1.sum(axis=1)
data_dc2['mode']=df_dc2.sum(axis=1)
df_dc1.head()
data_dc1['pick']=data_dc1['mode']>data_dc2['mode']
data_dc2['pick']=data_dc2['mode']>data_dc1['mode']

datac1=data_dc1['pick'] == True
data_dc1=data_dc1[datac1]
datac2=data_dc2['pick'] == True
data_dc2=data_dc2[datac2]
#(data.shape,data_dc1.shape,data_dc2.shape)
data_dc1.head()

host=[(i.split('=')[2]).rstrip('}') for i in column_list_datadc1]
host.append('mode')
host.append('pick')
data_dc1.columns=host


host=[(i.split('=')[2]).rstrip('}') for i in column_list_datadc2]
host.append('mode')
host.append('pick')
data_dc2.columns=host
data_dc2.head()

data_dc1=data_dc1.reindex_axis(sorted(data_dc1.columns), axis=1)
data_dc1.head()
data_dc2=data_dc2.reindex_axis(sorted(data_dc2.columns), axis=1)
data_dc2.head()

frames= [data_dc1,data_dc2]
data_dc = pd.concat(frames)
data_dc = data_dc.drop(labels=['pick','mode'], axis=1)

columnCount = len(data_dc.columns)
rankLevel = topNodes/columnCount

ts = data_dc[data_dc.rank(axis=1, ascending=False, pct=True) <= rankLevel].fillna(0).ix[:-1]

ts.head()
#data_dc1=data_dc1[pick_dc1]
#data_dc2=data_dc2[pick_dc2]
#data_dc1.tail()

/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,db1-1,db1-2,db1-3,db1-4,db1-5,db1-6,db1-7,db1-8
Timestamp,,,,,,,,
2016-11-12,18.775999,24.430000,11.475000,13.381000,12.664000,11.874000,10.318000,0.0
2016-11-13,18.594999,17.063000,17.139000,21.666000,10.282000,16.195000,17.969999,0.0
2016-11-14,36.029999,57.861000,30.896999,24.773001,28.521999,34.383999,38.693001,0.0
2016-11-15,33.683998,67.723999,34.042999,36.500999,36.451000,38.233002,27.799000,0.0
2016-11-16,38.668999,59.347000,32.133999,34.533001,38.937000,31.337000,33.018002,0.0
